# Imports and setting up viz

NB : conda env1 on Mac, lam1env on spirit (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_routing import *

In [3]:
rivers = cfeature.NaturalEarthFeature('physical', 'rivers_lake_centerlines', '10m',edgecolor=(0, 0, 0, 0.3), facecolor='none')

# Load files

LAM output analysis, routing files.

Simu irr et simu no-irr.

## Sims

In [4]:
dir='/data/ptiengou/JZ_simu_outputs/routing_native'

In [ ]:
filename='{}/tcst7_long/tcst7_hydrographs_1980-2010.nc'.format(dir)
sim_noirr=xr.open_mfdataset(filename)
rename_dict = {
        'time_counter' : 'time',
        'routing_hydrographs_r' : 'hydrographs',
}
sim_noirr = sim_noirr.rename(rename_dict)

# Resample to monthly average
# sim_noirr = sim_noirr.resample(time='MS').mean()

#add unit m³/s to hydrographs
sim_noirr['hydrographs'].attrs['units'] = 'm³/s'

sim_noirr.attrs['name'] = 'no_irr'

sim_noirr

In [ ]:
filename='{}/tcst7irr/*diag_routing_r.nc'.format(dir)
sim_irr = xr.open_mfdataset(filename)

rename_dict = {
        'time_counter' : 'time',
        'routing_hydrographs_r' : 'hydrographs',
}
sim_irr = sim_irr.rename(rename_dict)

# Resample to monthly average (necessary for first part of sim)
# sim_irr = sim_irr.resample(time='MS').mean()

#add unit m³/s to hydrographs
sim_irr['hydrographs'].attrs['units'] = 'm³/s'

sim_irr.attrs['name']='irr'

sim_irr

In [ ]:
filename='{}/tcst7irr0.6_long/*_hydrographs_1980-2010.nc'.format(dir)
sim_irr_06 = xr.open_mfdataset(filename)

rename_dict = {
        'time_counter' : 'time',
        'routing_hydrographs_r' : 'hydrographs',
}
sim_irr_06 = sim_irr_06.rename(rename_dict)

# Resample to monthly average (necessary for first part of sim)
# sim_irr_06 = sim_irr_06.resample(time='MS').mean()

#add unit m³/s to hydrographs
sim_irr_06['hydrographs'].attrs['units'] = 'm³/s'

sim_irr_06.attrs['name']='irr'

sim_irr_06

## Obs

In [ ]:
filename = '../../../obs/streamflow/GRDC-Monthly_Spain.nc'
# filename='../../../routing/GRDC_Monthly_Jan20_v1_ES.nc'
obs = xr.open_dataset(filename)
obs

In [ ]:
#create a subdataset from obs with stations that are in dict stations_dict_filtered
obs_filtered = obs.sel(id=(list(stations_dict_filtered.keys())))
obs_filtered

# River outlets and stations

format :     id: {'name':,  'river':, 'lon_grid':, 'lat_grid':, 'year_min':, 'year_max': },


In [ ]:
river_coords_merit = {
        1:{'name':'Ebro','river':'Ebro', 'lon_grid': 0.86, 'lat_grid': 40.725, 'year_min':2010, 'year_max':2022 },
        2:{'name':'Minho','river':'Minho', 'lon_grid': -8.875, 'lat_grid': 41.86, 'year_min':2010, 'year_max':2022 },
        3:{'name':'Tajo','river':'Tajo', 'lon_grid': -9.175, 'lat_grid': 38.6875, 'year_min':2010, 'year_max':2022 },
        4:{'name':'Duero','river':'Duero', 'lon_grid': -8.675, 'lat_grid': 41.1375, 'year_min':2010, 'year_max':2022 },
        5:{'name':'Guadiana','river':'Guadiana', 'lon_grid': -7.39, 'lat_grid': 37.175, 'year_min':2010, 'year_max':2022 },
        6:{'name':'Guadalquivir','river':'Guadalquivir', 'lon_grid': -6.325, 'lat_grid': 36.91, 'year_min':2010, 'year_max':2022 }
}

In [21]:
stations_merit = {
    6298992: {'name': 'Albarreal De Tajo',  'river': 'Tajo', 'lon_grid':-4.17499303817749, 'lat_grid':39.891666412353516, 'year_min':2010, 'year_max':2021 },
    # 6298249: {'name': 'Zaragoza',  'river': 'Ebro', 'lon_grid':-0.8749926686286926, 'lat_grid':41.65833282470703, 'year_min':2010, 'year_max':2021},
    # 6298481: {'name': 'San Miguel del Pino', 'river': 'Duero', 'lon_grid':-4.92499303817749, 'lat_grid':41.508331298828125, 'year_min':2010, 'year_max':2021},
    6298259:{'name': 'Badajoz', 'river': 'Guadiana', 'lat_grid': 38.85833358764648, 'lon_grid': -7.008326530456543, 'last_record': '2013-09-15', 'year_min':2010, 'year_max':2021},
    6298564: {'name': 'Villanueva de Azoague', 'lon_grid':-5.641659736633301, 'lat_grid':41.974998474121094, 'year_min':2010, 'year_max':2010}
}

# Maps

In [ ]:
ds=sim_irr
var='hydrographs'
map_ave(ds, var, cmap=wet, hex=True)

In [ ]:
#diff
ds1=sim_irr
ds2=sim_noirr
var='hydrographs'
map_diff_ave(ds1, ds2, var)

In [ ]:
#Display map of var for ds on restricted area
var='hydrographs'
ds = sim_irr

subdomain = {
    'latmin': 36.75,
    'latmax': 37,
    'lonmin': -6.5,
    'lonmax': -6.25
}
subdomain = subdomain_spain

#restrict ds to latmin, latmax, lonmin, lonmax
ds=restrict_ds(ds, subdomain)
plotvar = ds[var].mean(dim='time')
map_plotvar(plotvar, cmap=reds)

# Time series

## Stations

In [ ]:
fig=plt.figure(figsize=(10, 6))
ax = plt.axes()
# station_id, station_data=next(iter(stations_merit.items()))
station_id=6226100

ts_station(obs, ax, station_id)

In [ ]:
#plot a time series for each station from station_dict_filtered
fig, axes= plt.subplots(4,4, figsize=(40,25))
axes= axes.flatten()
ds_list=[sim_noirr, sim_irr]
for i, (station_id, station_data) in enumerate(stations_dict_filtered.items()):
    ts_with_obs(ds_list, obs, axes[i], station_id, station_data, year_min=2010, year_max=2022)
    ts_station(obs,axes[i], station_id, name=station_data['name'], year_min=2010, year_max=2022)

In [ ]:
#plot a seasonnal cycle for each station from station_dict_filtered
fig, axes= plt.subplots(4,4, figsize=(40,25))
axes= axes.flatten()
ds_list=[sim_noirr, sim_irr]
for i, (station_id, station_data) in enumerate(stations_dict_filtered.items()):
    sc_with_obs(ds_list, obs, axes[i], station_id, station_data, year_min=2010, year_max=2022)
    sc_station(obs,axes[i], station_id, name=station_data['name'], year_min=2010, year_max=2022)

## Spatial averaging

In [ ]:
var='hydrographs'

ds1=sim_irr
# ds2=ORCirr
ds_list=[ds1, ds2]
ds_list=[ds1]

year_max=2015
time_series_ave(ds_list, var, year_max=year_max, title=None)
seasonal_cycle_ave(ds_list, var, title=None)

## En un point lon-lat (ou plusieurs avec dict)

In [ ]:
#Time series
var="hydrographs"
ds1=sim_noirr
ds_list=[ds1]
year_min=2010
year_max=2015
lon=-6.325
lat=36.91

time_series_lonlat(ds_list, var, lon, lat, year_min=year_min, year_max=year_max, title=None)

In [ ]:
ds_list=[sim_noirr]
discharge_coord_ts(ds_list, stations_merit, var='hydrographs', year_min=2010, year_max=2015)

In [ ]:
ds_list=[sim]
discharge_coord_sc(ds_list, river_coords_merit, var='hydrographs', year_min=2010, year_max=2015)

# Metrics

## Compute and display in table

In [ ]:
station_id, station = next(iter(stations_dict_filtered.items()))
metric_list=[metric_sim_module, metric_obs_module, metric_bias, metric_rmse, metric_tcorr, metric_nse, metric_kge]
for metric_to_use in metric_list:
    metric_value=compute_metric_station(sim_noirr, obs, station_id, station, metric_to_use)
    print('{} for station {} : {}'.format(metric_to_use.__short_name__, station['name'], metric_value))

In [ ]:
# output a pandas dataframe with all metric values for a given list of metrics and stations
sim=sim_noirr
metric_list=[metric_sim_module, metric_obs_module, metric_bias, metric_rmse, metric_tcorr, metric_nse, metric_kge]
stations_dict=stations_dict_filtered
# define dataframe with one row per station and one column per metric
df=[]
for station_id, station in stations_dict.items():
    df.append({'Station':station['name']})
    for metric in metric_list:
        name=metric.__short_name__
        metric_value=compute_metric_station(sim, obs, station_id, station, metric)
        metric_value=np.round(metric_value, 2)
        # append metric_value to df
        df[-1][name]=metric_value

# convert df to pandas dataframe
df_noirr=pd.DataFrame(df)
# set Station as index
df_noirr.set_index('Station', inplace=True)
df_noirr['Bias (%)'] = np.round(df_noirr['Bias (m³/s)'] / df_noirr['Module (obs, m³/s)'], 3) * 100
#move Bias (%) to the 4th column
cols = df_noirr.columns.tolist()
cols = cols[:3] + cols[-1:] + cols[3:-1]
df_noirr = df_noirr[cols]
df_noirr

In [ ]:
# output a pandas dataframe with all metric values for a given list of metrics and stations
sim=sim_irr
metric_list=[metric_sim_module, metric_obs_module, metric_bias, metric_rmse, metric_tcorr, metric_nse, metric_kge]
stations_dict=stations_dict_filtered
# define dataframe with one row per station and one column per metric
df=[]
for station_id, station in stations_dict.items():
    df.append({'Station':station['name']})
    for metric in metric_list:
        name=metric.__short_name__
        metric_value=compute_metric_station(sim, obs, station_id, station, metric)
        metric_value=np.round(metric_value, 2)
        # append metric_value to df
        df[-1][name]=metric_value

# convert df to pandas dataframe
df_irr=pd.DataFrame(df)
# set Station as index
df_irr.set_index('Station', inplace=True)
df_irr['Bias (%)'] = np.round(df_irr['Bias (m³/s)'] / df_irr['Module (obs, m³/s)'], 3) * 100
#move Bias (%) to the 4th column
cols = df_irr.columns.tolist()
cols = cols[:3] + cols[-1:] + cols[3:-1]
df_irr = df_irr[cols]
df_irr

In [ ]:
df_noirr.describe()

In [ ]:
df_irr.describe()

## Display on map

In [ ]:
sim=sim_noirr
metric_to_use = metric_bias
display_metric_map(sim, obs, stations_dict_filtered, metric_to_use, metric_min=-100, metric_max=100, legend=True)

In [ ]:
sim=sim_irr
metric_to_use = metric_bias
display_metric_map(sim, obs, stations_dict_filtered, metric_to_use, metric_min=-100, metric_max=100, legend=True)

In [ ]:
sim=sim_noirr
metric_list=[metric_rmse, metric_tcorr, metric_nse, metric_kge]
for metric_to_use in metric_list:
    display_metric_map(sim, obs, stations_dict_filtered, metric_to_use, legend=True)

In [ ]:
sim=sim_irr
metric_list=[metric_rmse, metric_tcorr, metric_nse, metric_kge]
for metric_to_use in metric_list:
    display_metric_map(sim, obs, stations_dict_filtered, metric_to_use, legend=True)